In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
dataset = pd.read_csv('iris.csv')

In [3]:
features=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Species"]

test_indices = []
test_indices.append(random.randint(0, 49))
test_indices.append(random.randint(50, 99))
test_indices.append(random.randint(100, 149))

# print(test_indices)

train_indices = []
for i in range(0, 150):
    if i not in test_indices:
        train_indices.append(i)
        
test = dataset.loc[test_indices].drop(columns=["Id", "Species"])

train = dataset.loc[train_indices].drop(columns=["Id"])
test

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
11,4.8,3.4,1.6,0.2
85,6.0,3.4,4.5,1.6
132,6.4,2.8,5.6,2.2


In [4]:
# Calculating Mean vector for each class

#----------------------- Splitting train dataset according to the classes ----------------------------------------------

num = list(train.columns[0:4])

train_setosa = train[train['Species'] == 'Iris-setosa'][num]
# train_setosa
train_versicolor = train[train['Species'] == 'Iris-versicolor'][num]
# train_versicolor
train_virginica = train[train['Species'] == 'Iris-virginica'][num]
# train_virginica

#----------------------- Finding the mean vector for each class ---------------------------------------------------------

mean_setosa = list(train_setosa.mean())
# print(mean_setosa)
mean_versicolor = list(train_versicolor.mean())
mean_virginica = list(train_virginica.mean())

#----------------------- Finding the covariance matrix for each class ---------------------------------------------------

diff = train_setosa - mean_setosa
cov_setosa = np.dot(np.array(diff.T), np.array(diff)) / (train_setosa.shape[0] - 1)

diff1 = train_versicolor - mean_versicolor
cov_versicolor = np.dot(np.array(diff1.T), np.array(diff1)) / (train_versicolor.shape[0] - 1)

diff2 = train_virginica - mean_virginica
cov_virginica = np.dot(np.array(diff2.T), np.array(diff2)) / (train_virginica.shape[0] - 1)

means = [mean_setosa, mean_versicolor, mean_virginica]
covar_mat = [cov_setosa, cov_versicolor, cov_virginica]

#---------------------- Calculating Mahalanobis distance -----------------------------------------------------------------

dist = []

l = {0 : 0,
     1 : 51,                       # Dictionary for mapping the covar_mat, means indices to the actual classes
     2 : 102}

for i in range(test.shape[0]):
    x = []
    for j in range(3):
        mat1 = np.transpose(test.iloc[i] - means[j])
        mat2 = np.asarray(test.iloc[i] - means[j])
        mat3 = np.linalg.inv(covar_mat[j])
        res = np.sqrt(np.matmul(np.matmul(mat1, mat3), mat2))
        x.append([j, res])
    dist.append(x)
    z = sorted(x, key = lambda k : k[1])
    y = z[0][0]
    
    if i == 0:
        print(f"Actual Class : 'Iris-setosa' \t\t Predicted Class : {train['Species'][l[y]]}")
    if i == 1:
        print(f"Actual Class : 'Iris-versicolor' \t Predicted Class : {train['Species'][l[y]]}")
    if i == 2:
        print(f"Actual Class : 'Iris-virginica' \t Predicted Class : {train['Species'][l[y]]}")      
        
print('Mahalanobis Distance : ')

for i in range(3):
    print(f"Test Point 1 : {dist[i]}")

Actual Class : 'Iris-setosa' 		 Predicted Class : Iris-setosa
Actual Class : 'Iris-versicolor' 	 Predicted Class : Iris-versicolor
Actual Class : 'Iris-virginica' 	 Predicted Class : Iris-virginica
Mahalanobis Distance : 
Test Point 1 : [[0, 1.4690832755297831], [1, 9.927139567979395], [2, 12.436318019963709]]
Test Point 1 : [[0, 19.63674551929401], [1, 2.540088990888819], [2, 3.8211462252951676]]
Test Point 1 : [[0, 27.77109675774569], [1, 5.844301612271367], [2, 1.361077642861038]]
